## HIV News articles extraction from The Daily Observer. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for HIV News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls = [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'HIV site:www.observerbd.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

218 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="twitter:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(article.meta_description.strip())
            except:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(article.authors.strip())
            except:
                authors.append(soup.select_one('meta[name="cXenseParse:tss-byline"]')['content'].strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(str(article.publish_date))
            except:
                dates.append(soup.select_one('meta[name="cXenseParse:recs:publishtime"]')['content'].strip())
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(article.text.strip())
            except:
                news.append(soup.select_one('div#content').text.replace('\n', '').strip())
        except:
            news.append(None)
            
        # Extracts Keywords and Summaries    
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 15min 11sserverbd.com/2015/11/28/123131.phpb_comment_id=1325795027501456_1326639780750314


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls))

218 218 218 218 218 218 218 218


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,})
#tbl = tbl.dropna()
tbl.to_csv('The_Daily_Observer.csv', index=False)
tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Articles,Keywords,Summaries
0,London HIV patient becomes world's second AIDS...,,None,None,"London, Mar 5: An HIV-positive man in Britain ...","[worlds, hiv, aids, patient, gupta, man, known...","We can't detect anything,"" said Ravindra Gupta..."
1,Daily Observer,The Daily Observer is one of the most reliable...,None,None,"London, Mar 5: An HIV-positive man in Britain ...","[cleared, daily, worldwide, mar, man, adult, k...","London, Mar 5: An HIV-positive man in Britain ..."
2,HIV used to cure 'bubble boy' disease,,None,None,HIV used to cure 'bubble boy' disease\n\nResul...,"[used, hiv, immune, bone, st, cure, disease, b...",HIV used to cure 'bubble boy' diseaseResults o...
3,Some Rohingyas with HIV identified,,None,None,Many Rohingya refugee children and adults have...,"[diseases, hiv, hospital, treatment, refugee, ...",When asked about the status of the Rohingya ch...
4,HIV's ability to cause AIDS is weakening over ...,"Rapid evolution of HIV, the human immunodefici...",None,2014-12-02 00:00:00,HIV's ability to cause AIDS is weakening over ...,"[virulent, aids, hiv, cause, weakening, scient...",HIV's ability to cause AIDS is weakening over ...
